# DMN Moddle

This notebook demonstrates how to use `dmn-moddle` from Python in JupyterLite.

The library is loaded via a BroadcastChannel bridge to the JupyterLab extension.

## Load dmn-moddle

Run this cell to load the dmn-moddle library via the extension bridge.

In [ ]:
import operaton

DmnModdle = await operaton.load_dmn_moddle()
print(f"dmn-moddle loaded: {DmnModdle}")

## Parse DMN XML

Use the DmnModdle instance to parse DMN 1.3 XML.

In [ ]:
# Sample DMN XML (Decision Table)
dmn_xml = '''<?xml version="1.0" encoding="UTF-8"?>
<definitions xmlns="https://www.omg.org/spec/DMN/20191111/MODEL/"
             xmlns:dmndi="https://www.omg.org/spec/DMN/20191111/DMNDI/"
             xmlns:dc="http://www.omg.org/spec/DMN/20180521/DC/"
             id="sample-definitions"
             name="Sample Definitions"
             namespace="http://camunda.org/schema/1.0/dmn">
  <decision id="Decision_1" name="Approval Decision">
    <decisionTable id="DecisionTable_1">
      <input id="Input_1" label="Amount">
        <inputExpression id="InputExpression_1" typeRef="number">
          <text>amount</text>
        </inputExpression>
      </input>
      <output id="Output_1" label="Approved" name="approved" typeRef="boolean" />
      <rule id="Rule_1">
        <inputEntry id="InputEntry_1">
          <text>&lt; 1000</text>
        </inputEntry>
        <outputEntry id="OutputEntry_1">
          <text>true</text>
        </outputEntry>
      </rule>
      <rule id="Rule_2">
        <inputEntry id="InputEntry_2">
          <text>&gt;= 1000</text>
        </inputEntry>
        <outputEntry id="OutputEntry_2">
          <text>false</text>
        </outputEntry>
      </rule>
    </decisionTable>
  </decision>
</definitions>'''

print("DMN XML loaded")

In [ ]:
result = await operaton.parse_dmn(dmn_xml)
definitions = result.rootElement

print(f"Parsed definitions: {definitions.id}")
print(f"Name: {definitions.name}")

In [ ]:
# Access decision elements
for element in definitions.drgElement:
    print(f"DRG element: {element.id} ({element['$type']})")
    
    if hasattr(element, 'decisionLogic'):
        decision_table = element.decisionLogic
        print(f"  Decision table: {decision_table.id}")
        
        if hasattr(decision_table, 'input'):
            for inp in decision_table.input:
                print(f"    Input: {inp.id} - {getattr(inp, 'label', '(no label)')}")
        
        if hasattr(decision_table, 'output'):
            for out in decision_table.output:
                print(f"    Output: {out.id} - {getattr(out, 'label', '(no label)')}")
        
        if hasattr(decision_table, 'rule'):
            print(f"    Rules: {len(list(decision_table.rule))}")

## Serialize to XML

Convert the parsed DMN model back to XML.

In [ ]:
xml_output = await operaton.to_dmn_xml(definitions)
print(xml_output[:500] + '...')